# Script para generar la solución del Tercer Benchmark de la Competencia

## Si no presentaste aún tu primera solución, tenes la oportunidad de hacerlo en pocos Clicks!

**Hola! **  
  
Este Script es un Ejemplo de Procesamiento de los Datos, Modelado y Generación de una Solución.

Agregamos una pequeña explicación de lo que se hace en cada paso para ayudar a los que están comenzando ahora

### Importamos las librerías que vamos a utilizar

In [1]:
import pandas as pd

### Lectura de las Bases

Observamos los datos que tenemos disponibles en https://www.kaggle.com/c/interbank-internacional-2019/data

Vamos a trabajar ahora con todas las bases disponibles

In [2]:
X_test = pd.read_csv("data/raw/ib_base_inicial_test/ib_base_inicial_test.csv")
sunat = pd.read_csv("data/raw/ib_base_sunat/ib_base_sunat.csv")
train = pd.read_csv("data/raw/ib_base_inicial_train/ib_base_inicial_train.csv")
rcc = pd.read_csv("data/raw/ib_base_rcc/ib_base_rcc.csv")
reniec = pd.read_csv("data/raw/ib_base_reniec/ib_base_reniec.csv")
digital = pd.read_csv("data/raw/ib_base_digital/ib_base_digital.csv")
vehicular = pd.read_csv("data/raw/ib_base_vehicular/ib_base_vehicular.csv")
campanias = pd.read_csv("data/raw/ib_base_campanias/ib_base_campanias.csv")

### Creación del Target de predicción

Consutruimos un target continuo para detectar quienes son clientes rentables y, por tanto, es conveniente hacerles campaña para atraerlos.

In [3]:
y_train = train[['codmes', 'id_persona', 'margen']].copy()
y_train["prediction_id"] = y_train["id_persona"].astype(str) + "_" + y_train["codmes"].astype(str)
# y_train["target"] = y_train["margen"].astype("float32")
y_train = y_train.set_index("prediction_id")
X_train = train.drop(["codtarget", "margen"], axis=1)
X_train["prediction_id"] = X_train["id_persona"].astype(str) + "_" + X_train["codmes"].astype(str)
del train

### Creación de Variables

Primero creamos una variable en base a las columnas que tenemos en nuestra base de train

Vamos a crear la variable "ratio" que va a ser el cociente entre la linea ofrecida y el ingreso neto

In [4]:
X_train["ratio"] = X_train["linea_ofrecida"] / X_train["ingreso_neto"]
X_test["ratio"] = X_test["linea_ofrecida"] / X_test["ingreso_neto"]

Vamos a trabajar ahora con la base de **RCC**:
* Imputamos los valores perdidos con el valor "-1"
* Agrupamos por mes y persona las distintas informaciones para luego agregarla a nuestra base central (train)

In [5]:
rcc.clasif.fillna(-1, inplace=True)
rcc.rango_mora.fillna(-1, inplace=True)
rcc_clasif = rcc.groupby(["codmes", "id_persona"]).clasif.max().reset_index().set_index("codmes").sort_index().astype("int32")
rcc_mora = rcc.groupby(["codmes", "id_persona", "rango_mora"]).mto_saldo.sum().unstack(level=2, fill_value=0).reset_index().set_index("codmes").sort_index().astype("int32")
rcc_producto = rcc.groupby(["codmes", "id_persona", "producto"]).mto_saldo.sum().unstack(level=2, fill_value=0).reset_index().set_index("codmes").sort_index().astype("int32")
rcc_banco = rcc.groupby(["codmes", "id_persona", "cod_banco"]).mto_saldo.sum().unstack(level=2, fill_value=0).reset_index().set_index("codmes").sort_index().astype("int32")
del rcc

In [6]:
rcc_mora.head()

rango_mora,id_persona,-1.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0
codmes,,,,,,,,,
201712,2,25707,10222,0,0,0,0,0,0
201712,3,0,1013,0,0,0,0,0,0
201712,5,0,7565,0,0,0,0,0,0
201712,8,0,13954,0,0,0,0,0,0
201712,16,1460,7300,0,0,0,0,0,0


Renombramos las columnas para que sean más explicativas de lo que representan

In [7]:
rcc_mora.columns = ["mora_" + str(c) if c != "id_persona" else c for c in rcc_mora.columns ]
rcc_producto.columns = ["producto_" + str(c) if c != "id_persona" else c for c in rcc_producto.columns]
rcc_banco.columns = ["banco_" + str(c) if c != "id_persona" else c for c in rcc_banco.columns]

Vamos a trabajar luego con la base de **Campañas**:
* Agrupamos por mes y persona las distintas informaciones para poder sumarlas a nuestra base train

In [8]:
camp_canal = campanias.groupby(["codmes", "id_persona", "canal_asignado"]).size().unstack(level=2, fill_value=0).reset_index().set_index("codmes").sort_index().astype("int32")
camp_prod = campanias.groupby(["codmes", "id_persona", "producto"]).size().unstack(level=2, fill_value=0).reset_index().set_index("codmes").sort_index().astype("int32")
#del campanias

In [13]:
camp_canal[camp_canal.id_persona == 237325]


canal_asignado,id_persona,ABP,AC_PROV,ATM/BPI,BANCA TELEFONICA,BOLSA,BOLSA(E),BOLSA(EX),BOLSA(EX-E),BOLSA(EX-P),...,RED_DE_TIENDAS,Red de Tiendas,S1P,SOMOS_UNO_PROVINCIA,TELEVENTAS,TLV,TLV BD,TLV CD,TLV SEGMENTOS,TLV(EX)
codmes,,,,,,,,,,,,,,,,,,,,,
201801,237325,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201802,237325,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201803,237325,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201805,237325,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
201806,237325,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
201808,237325,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201809,237325,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201901,237325,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [29]:
camp_canal_agg =camp_canal.loc[meses[201902]].groupby("id_persona").sum().reset_index()

In [30]:
camp_canal_agg[camp_canal_agg.id_persona == 237325]


canal_asignado,id_persona,ABP,AC_PROV,ATM/BPI,BANCA TELEFONICA,BOLSA,BOLSA(E),BOLSA(EX),BOLSA(EX-E),BOLSA(EX-P),...,RED_DE_TIENDAS,Red de Tiendas,S1P,SOMOS_UNO_PROVINCIA,TELEVENTAS,TLV,TLV BD,TLV CD,TLV SEGMENTOS,TLV(EX)
167665,237325,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
meses[201902]

slice(201809, 201811, None)

Renombramos las columnas para que sean más explicativas de lo que representan

In [12]:
camp_canal.columns = ["canal_" + str(c) if c != "id_persona" else c for c in camp_canal.columns]
camp_prod.columns = ["producto_campania_" + str(c) if c != "id_persona" else c for c in camp_prod.columns]

Vamos a trabajar luego con la base de **Digital**:
* Renombramos la columna codday por codmes y la colocamos como índice
* Agrupamos por mes y persona las distintas informaciones para poder sumarlas a nuestra base train

In [13]:
digital["codmes"] = digital.codday.astype(str).str[:-2].astype(int)
digital = digital.drop("codday", axis=1).fillna(0)
digital = digital.groupby(["codmes", "id_persona"]).sum().reset_index().set_index("codmes").sort_index().astype("int32")

Hacemos lo mismo que antes con las bases **Sunat**, **Vehicular** y **Reniec**

In [15]:
sunat = sunat.groupby(["id_persona", "activ_econo"]).meses_alta.sum().unstack(level=1, fill_value=0).astype("int32")
vehicular1 = vehicular.groupby(["id_persona", "marca"]).veh_var1.sum().unstack(level=1, fill_value=0).astype("float32")
vehicular2 = vehicular.groupby(["id_persona", "marca"]).veh_var2.sum().unstack(level=1, fill_value=0).astype("float32")
reniec = reniec.set_index("id_persona").astype("float32")
del vehicular

In [16]:
vehicular1.columns = [c + "_v1" for c in vehicular1.columns]
vehicular2.columns = [c + "_v2" for c in vehicular2.columns]

Agregamos después las columnas calculadas a nuestra base de Train

In [17]:
X_train = X_train.set_index("prediction_id").astype("int32").reset_index().set_index("id_persona").join(vehicular1).join(vehicular2).join(reniec).join(sunat)
X_test = X_test.set_index("prediction_id").astype("int32").reset_index().set_index("id_persona").join(vehicular1).join(vehicular2).join(reniec).join(sunat)
del vehicular1, vehicular2, reniec, sunat

In [18]:
import gc
gc.collect()

1277

Agregamos las columnas de las demás bases en los meses correspondientes a nustra base de Train

In [19]:
meses = {
    201901: slice(201808, 201810),
    201902: slice(201809, 201811),
    201903: slice(201810, 201812),
    201904: slice(201811, 201901),
    201905: slice(201812, 201902),
    201906: slice(201901, 201903),
    201907: slice(201902, 201904)
}

meses_train = X_train.codmes.unique()
meses_test = X_test.codmes.unique()
complementos = []
for mes in meses.keys():
    print("*"*10, mes, "*"*10)
    res = pd.concat([
        rcc_clasif.loc[meses[mes]].groupby("id_persona").sum(),
        rcc_mora.loc[meses[mes]].groupby("id_persona").sum(),
        rcc_producto.loc[meses[mes]].groupby("id_persona").sum(),
        rcc_banco.loc[meses[mes]].groupby("id_persona").sum(),
        camp_canal.loc[meses[mes]].groupby("id_persona").sum(),
        camp_prod.loc[meses[mes]].groupby("id_persona").sum(),
        digital.loc[meses[mes]].groupby("id_persona").sum()
        
    ], axis=1)
    res["codmes"] = mes
    res = res.reset_index().set_index(["id_persona", "codmes"]).astype("float32")
    complementos.append(res)

gc.collect()
print("concatenando complementos")
complementos = pd.concat(complementos)
gc.collect()
print("X_train join")
X_train = X_train.reset_index().join(complementos, on=["id_persona", "codmes"]).set_index("prediction_id")
gc.collect()
print("X_test join")
X_test = X_test.reset_index().join(complementos, on=["id_persona", "codmes"]).set_index("prediction_id")
gc.collect()

del rcc_clasif, rcc_mora, rcc_producto, rcc_banco, camp_canal, camp_prod, digital, complementos,res
gc.collect()

********** 201901 **********
********** 201902 **********
********** 201903 **********
********** 201904 **********
********** 201905 **********
********** 201906 **********
********** 201907 **********
concatenando complementos
X_train join
X_test join


84

In [31]:
camp_canal

canal_asignado,id_persona,ABP,AC_PROV,ATM/BPI,BANCA TELEFONICA,BOLSA,BOLSA(E),BOLSA(EX),BOLSA(EX-E),BOLSA(EX-P),...,RED_DE_TIENDAS,Red de Tiendas,S1P,SOMOS_UNO_PROVINCIA,TELEVENTAS,TLV,TLV BD,TLV CD,TLV SEGMENTOS,TLV(EX)
codmes,,,,,,,,,,,,,,,,,,,,,
201801,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201801,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
201801,20,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
201801,29,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
201801,32,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201801,44,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
201801,63,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201801,75,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201801,77,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Renombrado de Variables con nombre no ascii
El algoritmo que usamos no se lleva bien con cadenas de texto con caracteres especiales, las renombramos.

In [20]:
for i, c in enumerate(X_train.columns[[not all(ord(c) < 128 for c in s) for s in X_train.columns]]):
    X_train["non_ascii_" + str(i)] = X_train[c]
    X_train = X_train.drop(c, axis= 1)
    X_test["non_ascii_" + str(i)] = X_test[c]
    X_test = X_test.drop(c, axis= 1)

### Entrenamiento del Modelo

Para entrenar nuestro modelo vamos a usar LightGBM

In [21]:
from lightgbm import LGBMRegressor
gc.collect()

14

In [22]:
drop_cols = ["codmes"]
test_preds = []
train_preds = []
y_train["target"] = y_train["margen"].astype("float32")
for mes in X_train.codmes.unique():
    print("*"*10, mes, "*"*10)
    Xt = X_train[X_train.codmes != mes]
    yt = y_train.loc[Xt.index, "target"]
    Xt = Xt.drop(drop_cols, axis=1)

    Xv = X_train[X_train.codmes == mes]
    yv = y_train.loc[Xv.index, "target"]
    
    learner = LGBMRegressor(n_estimators=1000)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="mae",
                eval_set=[(Xt, yt), (Xv.drop(drop_cols, axis=1), yv)], verbose=50)
    gc.collect()
    test_preds.append(pd.Series(learner.predict(X_test.drop(drop_cols, axis=1)),
                                index=X_test.index, name="fold_" + str(mes)))
    train_preds.append(pd.Series(learner.predict(Xv.drop(drop_cols, axis=1)),
                                index=Xv.index, name="probs"))
    gc.collect()

test_preds = pd.concat(test_preds, axis=1).mean(axis=1)
train_preds = pd.concat(train_preds)

********** 201902 **********
Training until validation scores don't improve for 10 rounds.
[50]	training's l2: 4004.73	training's l1: 20.0656	valid_1's l2: 4653.79	valid_1's l1: 19.8936
Early stopping, best iteration is:
[45]	training's l2: 4052.75	training's l1: 20.1509	valid_1's l2: 4650.94	valid_1's l1: 19.9442
********** 201904 **********
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[37]	training's l2: 4016.15	training's l1: 20.0293	valid_1's l2: 4936.57	valid_1's l1: 21.4454
********** 201901 **********
Training until validation scores don't improve for 10 rounds.
Early stopping, best iteration is:
[31]	training's l2: 4490.18	training's l1: 21.8321	valid_1's l2: 3666.27	valid_1's l1: 18.8463
********** 201903 **********
Training until validation scores don't improve for 10 rounds.
[50]	training's l2: 3787.19	training's l1: 18.8481	valid_1's l2: 5250.06	valid_1's l1: 23.3027
Early stopping, best iteration is:
[53]	training's l2: 3

In [23]:
from lightgbm import LGBMClassifier
gc.collect()

20

In [24]:
drop_cols = ["codmes"]
fi = []
test_probs = []
train_probs = []
y_train["target"] = (y_train["margen"] > 0).astype("int32")
for mes in X_train.codmes.unique():
    print("*"*10, mes, "*"*10)
    Xt = X_train[X_train.codmes != mes]
    yt = y_train.loc[Xt.index, "target"]
    Xt = Xt.drop(drop_cols, axis=1)

    Xv = X_train[X_train.codmes == mes]
    yv = y_train.loc[Xv.index, "target"]
    
    learner = LGBMClassifier(n_estimators=1000)
    learner.fit(Xt, yt,  early_stopping_rounds=10, eval_metric="mae",
                eval_set=[(Xt, yt), (Xv.drop(drop_cols, axis=1), yv)], verbose=50)
    gc.collect()
    test_probs.append(pd.Series(learner.predict_proba(X_test.drop(drop_cols, axis=1))[:, -1],
                                index=X_test.index, name="fold_" + str(mes)))
    train_probs.append(pd.Series(learner.predict_proba(Xv.drop(drop_cols, axis=1))[:, -1],
                                index=Xv.index, name="probs"))
    gc.collect()

test_probs = pd.concat(test_probs, axis=1).mean(axis=1)
train_probs = pd.concat(train_probs)

********** 201902 **********
Training until validation scores don't improve for 10 rounds.
[50]	training's binary_logloss: 0.256867	training's l1: 0.15232	valid_1's binary_logloss: 0.274808	valid_1's l1: 0.149752
[100]	training's binary_logloss: 0.246962	training's l1: 0.148699	valid_1's binary_logloss: 0.273296	valid_1's l1: 0.147766
Early stopping, best iteration is:
[104]	training's binary_logloss: 0.246386	training's l1: 0.14855	valid_1's binary_logloss: 0.273269	valid_1's l1: 0.147701
********** 201904 **********
Training until validation scores don't improve for 10 rounds.
[50]	training's binary_logloss: 0.25543	training's l1: 0.150584	valid_1's binary_logloss: 0.274822	valid_1's l1: 0.160573
[100]	training's binary_logloss: 0.244871	training's l1: 0.146613	valid_1's binary_logloss: 0.273606	valid_1's l1: 0.159015
Early stopping, best iteration is:
[120]	training's binary_logloss: 0.241704	training's l1: 0.145573	valid_1's binary_logloss: 0.273506	valid_1's l1: 0.158645
*********

In [25]:
test = pd.concat([test_probs.rename("probs"), test_preds.rename("preds")], axis=1)
train = pd.concat([train_probs.rename("probs"), train_preds.rename("preds")], axis=1)

### Optimización de punto de corte
Con las probabilidades calculadas en validación, calcularmos el punto de corte optimo para maximizar la ecuación económica de la empresa

In [26]:
from scipy.optimize import differential_evolution

def clasificar(res, c):
    return ((res.probs > c[0]) | (res.preds > c[1])) * c[2] + ((res.probs > c[3]) & (res.preds > c[4])) * c[5] > c[6]

def cost(res, coefs):
    return -((clasificar(res, coefs) * res.margen) / res.margen.sum()).sum()

res = y_train.join(train)
optimization = differential_evolution(lambda x: cost(res, x), [(-100, 100), (0, 1), (0, 1),
                                                               (-100, 100), (0, 1), (0, 1),
                                                               (0, 2)])
optimization

     fun: -1.1216502916273559
 message: 'Optimization terminated successfully.'
    nfev: 1478
     nit: 13
 success: True
       x: array([-62.34480716,   0.46702186,   0.13137919, -27.3907871 ,
         0.93077573,   0.99411005,   0.32557387])

### Guardado del modelo para hacer la presentación

Finalmente creamos el archivo CSV que podemos subir como nuestra Solución a la competencia

Empezá con este archivo y luego podes seguir mejorandolo a ver si subis en posiciones!

In [27]:
test_preds = clasificar(test, optimization["x"]).astype(int)
test_preds.index.name="prediction_id"
test_preds.name="class"
test_preds.to_csv("benchmark3.csv", header=True)